# Note:
-(make sure both data1 and data2 have the same number of numerical and categorical columns)
- Run all the cells for data1
- Then, run all the cells for data2
- Then after you have the correlation matrices for both (data1 and data2) --> run the difference between matrices test
- This is the only way this file will work. If you dont follow the steps you will get a result of ZERO

In [2]:
import numpy as np
import pandas as pd
from numpy import linalg
from sklearn.model_selection import train_test_split

# Upload Dataset

In [4]:
# Dataset before augmentation
data2 = pd.read_csv("/Users/elleemortensen/Documents/GitHub/BP24/Ellee/Data/Stacked/stacked_orig.csv", header=None)
data1 = pd.read_csv("/Users/elleemortensen/Documents/GitHub/BP24/Ellee/Data/Uniform/uniform_orig.csv", header=None)

# Dataset after augmentation
data3 = pd.read_csv("/Users/elleemortensen/Documents/GitHub/BP24/Ellee/Data/Stacked/stacked_new.csv", header=None)
# data2 = pd.read_csv("/Users/elleemortensen/Documents/GitHub/BP24/Ellee/Data/Uniform/uniform_new.csv", header=None)
# data2 = pd.read_csv("/Users/elleemortensen/Documents/GitHub/BP24/Ellee/Data/Gaussian/gaussian_orig.csv", header=None)
# data2 = pd.read_csv("/Users/elleemortensen/Documents/GitHub/BP24/Ellee/Data/Gaussian/gaussian_new.csv", header=None)



# Check Shapes of Datasets (they must be the same)

In [6]:
# Print shapes for debugging
print(f"Shape of data1: {data1.shape}")
print(f"Shape of data2: {data2.shape}")
print(f"Shape of data3: {data3.shape}")

Shape of data1: (343, 25)
Shape of data2: (240, 25)
Shape of data3: (240, 25)


In [ ]:
# If the datasets do not have the same columns ---> make one dataset match the other one (order does not matter)
# Subset columns so both have the same 
# (we need to make sure we have the same categorical and numerical columns) ---> you do this after this cell
data1 = data1.iloc[:, -13:]
data1.shape

# Convert columns into Categorical (Only for Uniform and Gaussian Distributions)

In [14]:
random_columns = [2,7,10,15]
print("Randomly selected columns:", random_columns)

# Create a copy of the saved_X_train variable
cat_X_train = X_train.copy()

# Process the randomly selected columns into categorical
for col in random_columns:
    cat_X_train[col] = X_train[col].round().astype(int).astype("category")
cat_X_train.dtypes

Randomly selected columns: [2, 7, 10, 15]


0      float64
1      float64
2     category
3      float64
4      float64
5      float64
6      float64
7     category
8      float64
9      float64
10    category
11     float64
12     float64
13     float64
14     float64
15    category
16     float64
17     float64
18     float64
19     float64
20     float64
21     float64
22     float64
23     float64
dtype: object

In [16]:
drop_X_train = cat_X_train.drop(labels=random_columns, axis=1)
drop_X_train.head() 

,0,1,3,4,5,6,8,9,11,12,13,14,16,17,18,19,20,21,22,23
0,0.448339,0.881950,0.997296,1.070843,0.774824,0.450895,0.948064,0.233751,0.884912,1.194469,0.639893,0.827687,0.441712,0.922309,0.970799,0.561568,0.884867,1.198576,0.983875,0.712703
1,0.471165,0.305296,0.398776,1.003245,0.974305,1.043214,0.910891,0.781484,0.679196,0.214840,0.815345,0.794381,0.302509,0.691533,0.693729,0.410874,0.324358,0.809678,0.574804,0.977890
2,0.619271,0.423921,0.445857,0.517032,1.181797,0.883740,0.850096,0.877157,0.365843,0.601060,0.935199,0.498430,0.838324,0.934942,0.482507,0.418814,1.000564,0.528500,0.827828,0.651538
3,0.192957,0.062524,0.831945,0.458847,0.858328,0.954374,0.549190,0.985553,0.264698,0.703247,0.680731,0.378326,0.099141,0.222815,0.157227,0.708536,0.529012,0.593757,0.621701,0.634567
4,0.820657,0.338822,0.068901,0.803221,0.408169,0.542404,0.098391,0.046909,0.040429,0.311103,0.598771,0.600084,0.223667,0.007614,0.604804,0.284894,0.433043,0.806125,0.387838,0.860067


In [18]:
# Create a copy of the saved_X_test variable
cat_X_test = X_test.copy()

# Process the randomly selected columns
for col in random_columns:
    cat_X_test[col] = X_test[col].round().astype(int).astype("category")

# Check if 4 columns were converted to categorical
print(cat_X_test.dtypes)

0      float64
1      float64
2     category
3      float64
4      float64
5      float64
6      float64
7     category
8      float64
9      float64
10    category
11     float64
12     float64
13     float64
14     float64
15    category
16     float64
17     float64
18     float64
19     float64
20     float64
21     float64
22     float64
23     float64
dtype: object


# Convert columns into Categorical (Only for Stacked Distribution)

In [24]:
for column in data2.columns[-9:]:
    data2[column] = data2[column].astype('category')

for column in data3.columns[-9:]:
    data3[column] = data3[column].astype('category')

# Concat Synthetic Dataset with New generated dataset

In [26]:
# Concat (adding rows)
combined_df = np.vstack((data2, data3))

# change numpy array into a dataframe
combined_data = pd.DataFrame(combined_df, columns=data2.columns)

# Convert the last number of columns to categorical (to make sure they have the same)
for column in combined_data.columns[-9:]:
    combined_data[column] = combined_data[column].astype('category')

# Verify the changes
print(combined_data.dtypes)

0      float64
1      float64
2      float64
3      float64
4      float64
5      float64
6      float64
7      float64
8      float64
9      float64
10     float64
11     float64
12     float64
13     float64
14     float64
15     float64
16    category
17    category
18    category
19    category
20    category
21    category
22    category
23    category
24    category
dtype: object


# Split Dataset for combined_data

In [30]:
# # Split dataset into X and y (data1)
# X = combined_data.iloc[:, :-1]  # All columns except the last one
# y = combined_data.iloc[:, -1]   # Only the last column


# # Split dataset into X_train and y_train
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = combined_data.iloc[:336,:-1]
X_test = combined_data.iloc[336:,:-1]
y_train = combined_data.iloc[:336,-1]
y_test = combined_data.iloc[336:,-1]

# Splitting X_train into numerical subset 
numerical_df = X_train.select_dtypes(include = ['float', 'float64'])

# Splitting X_train into categorical subset 
categorical_df = X_train.select_dtypes(exclude=['float', 'float64'])

# Split Dataset for Synthetic Data

In [12]:
# # data2
# X = data1.iloc[:, :-1]
# y = data1.iloc[:, -1]


# # Split dataset into X_train and y_train
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = data1.iloc[:240,:-1]
X_test = data1.iloc[240:,:-1]
y_train = data1.iloc[:240,-1]
y_test = data1.iloc[240:,-1]

# Splitting X_train into numerical subset 
numerical_df = X_train.select_dtypes(include = ['float', 'float64'])

# Splitting X_train into categorical subset 
categorical_df = X_train.select_dtypes(exclude=['float', 'float64'])

# Correlation between columns test (Matrix)

In [20]:
##################### Correlation between columns (numerical) Code ############################
# Takes the X_train data to find correlation between NUMERICAL features
def num_corr(X_train_numerical):
    matrix = X_train_numerical.corr(method='pearson')
    print("---------------------------Correlation Matrix------------------------- \n", matrix)
     
#Calls the function so the matrix prints out    
correlation_matrix = num_corr(numerical_df)


---------------------------Correlation Matrix------------------------- 
           0         1         2         3         4         5         6   \
0   1.000000  0.102176  0.101455  0.046385  0.100937  0.159250  0.156756   
1   0.102176  1.000000  0.069224  0.100464  0.100563  0.191945  0.150913   
2   0.101455  0.069224  1.000000  0.099960  0.042315  0.076310  0.073328   
3   0.046385  0.100464  0.099960  1.000000  0.104913  0.109463  0.138531   
4   0.100937  0.100563  0.042315  0.104913  1.000000  0.165329  0.041804   
5   0.159250  0.191945  0.076310  0.109463  0.165329  1.000000  0.134224   
6   0.156756  0.150913  0.073328  0.138531  0.041804  0.134224  1.000000   
7  -0.081726  0.027388  0.011466 -0.022537 -0.035847  0.145137  0.013968   
8   0.029898  0.124633  0.147564  0.129089  0.057620  0.219401  0.049445   
9   0.041401  0.081107 -0.003502  0.132483  0.124073  0.174872  0.160732   
10  0.020039  0.179525 -0.001106 -0.029936  0.083772  0.034538  0.113979   
11  0.140919  0

# Save Correlation Matrix into a Dataframe for combined_data

In [32]:
#combined_data
correlation_matrix = numerical_df.corr()
correlation_df_combined = pd.DataFrame(correlation_matrix)
print(correlation_df_combined)

correlation_df_combined.shape

          0         1         2         3         4         5         6   \
0   1.000000  0.119394  0.093576  0.165254  0.262304  0.092039  0.105108   
1   0.119394  1.000000  0.202749  0.233485  0.175705  0.170697  0.177917   
2   0.093576  0.202749  1.000000  0.279403  0.181573  0.156761  0.220407   
3   0.165254  0.233485  0.279403  1.000000  0.209248  0.190193  0.124189   
4   0.262304  0.175705  0.181573  0.209248  1.000000  0.250837  0.205605   
5   0.092039  0.170697  0.156761  0.190193  0.250837  1.000000  0.005947   
6   0.105108  0.177917  0.220407  0.124189  0.205605  0.005947  1.000000   
7   0.212927  0.106450  0.128576  0.070611  0.181345  0.040819  0.164350   
8   0.227360  0.184617  0.232800  0.240305  0.264772  0.185244  0.234702   
9   0.208547  0.231368  0.251332  0.211879  0.179882  0.175152  0.215160   
10  0.221155  0.215195  0.174298  0.296583  0.170258  0.181904  0.254341   
11  0.188710  0.236309  0.257441  0.248219  0.192283  0.060425  0.295934   
12  0.164961

(16, 16)

# Save Correlation Matrix into a Dataframe for Synthetic dataset

In [22]:
#data1
correlation_matrix = numerical_df.corr()
correlation_df1 = pd.DataFrame(correlation_matrix)
print(correlation_df1)

correlation_df1.shape

          0         1         2         3         4         5         6   \
0   1.000000  0.102176  0.101455  0.046385  0.100937  0.159250  0.156756   
1   0.102176  1.000000  0.069224  0.100464  0.100563  0.191945  0.150913   
2   0.101455  0.069224  1.000000  0.099960  0.042315  0.076310  0.073328   
3   0.046385  0.100464  0.099960  1.000000  0.104913  0.109463  0.138531   
4   0.100937  0.100563  0.042315  0.104913  1.000000  0.165329  0.041804   
5   0.159250  0.191945  0.076310  0.109463  0.165329  1.000000  0.134224   
6   0.156756  0.150913  0.073328  0.138531  0.041804  0.134224  1.000000   
7  -0.081726  0.027388  0.011466 -0.022537 -0.035847  0.145137  0.013968   
8   0.029898  0.124633  0.147564  0.129089  0.057620  0.219401  0.049445   
9   0.041401  0.081107 -0.003502  0.132483  0.124073  0.174872  0.160732   
10  0.020039  0.179525 -0.001106 -0.029936  0.083772  0.034538  0.113979   
11  0.140919  0.095028  0.031245  0.081945  0.214767  0.165693  0.037640   
12  0.046418

(24, 24)

# Differences in Matrices (Frobenius Norm)

In [34]:
# Convert the dataframes to numpy arrays
matrix1 = correlation_df1.to_numpy()
matrix2 = correlation_df_combined.to_numpy()

# Compute the Frobenius norm of the difference between the matrices. 
frobenius_abs = np.linalg.norm(matrix1 - matrix2, ord='fro')   # Absolute error with Frobenius norm

frobenius_rel = frobenius_abs/np.linalg.norm(matrix1, ord='fro')    # Relative error with Frobenius norm

print(f"Frobenius norm difference (absolute error) : {frobenius_abs: .3f}")
print(f"Frobenius notrelative error : {frobenius_rel: .3f}")


ValueError: operands could not be broadcast together with shapes (24,24) (16,16) 

# Differences in Arrays (L2 Norm)

In [ ]:
#array1 ...
#array2 ...

# Compute the L2 norm (Euclidean norm) of the difference between the arrays
L2_norm = np.linalg.norm(array1 - array2, ord=2)

print(f"L2 norm difference: {L2_norm: .3f}")
